In [1]:
import re
import pandas as pd
import numpy as np
import csv
import nltk
from nltk.corpus import stopwords

In [2]:
dataset = 2

In [3]:
if dataset == 1:
    train = pd.read_csv('C:/Users/eric the cool/Desktop/9665/project/SMS_train.csv', encoding = 'latin-1')
    test = pd.read_csv('C:/Users/eric the cool/Desktop/9665/project/SMS_test.csv', encoding = 'latin-1')
    train['Label'] =  train['Label'].map({"Non-Spam":0,"Spam":1})
    test['Label'] =  test['Label'].map({"Non-Spam":0,"Spam":1})

if dataset == 2:
    data = pd.read_csv('C:/Users/eric the cool/Desktop/9665/project/spam_ham_dataset.csv', encoding = 'latin-1')
    data = data.rename(columns={'label_num': 'Label', 'text': 'Message_body'})
    
    from sklearn.model_selection import train_test_split
    train, test = train_test_split(data, test_size=0.2, random_state=0)
    train = train[['Message_body', 'Label']]
    test = test[['Message_body', 'Label']]
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)

In [4]:
def preprocessing (text):
    
    ps = nltk.stem.PorterStemmer()
    # Normalization and cleaning
    text = text.lower()
    text = re.sub("(http|https|www)(:|\.)\S+.com"," ",text)
    text = re.sub('[^a-zA-Z0-9\n]', ' ', text)
    text = re.sub("[^\w\d]"," ",text)
    text = re.sub("\d+"," ",text)
    text = re.sub('\s+',' ', text)
    text = " ".join([ps.stem(t) for t in text.split() if t not in nltk.corpus.stopwords.words("english")])
    
    # Tokenization and Stemming
#     token_text = []
#     ps = nltk.stem.PorterStemmer()
#     for word in nltk.word_tokenize(text):
#         token_text.append(ps.stem(word))
        
    return text

In [5]:
processed_text = []
for i in range(train.shape[0]):
    processed_text.append(preprocessing(train.loc[i].at["Message_body"]))
    
train['processed_text'] = processed_text

In [6]:
processed_text2 = []
for i in range(test.shape[0]):
    processed_text2.append(preprocessing(test['Message_body'][i]))
    
test['processed_text'] = processed_text2

In [11]:
def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features

In [7]:
train['processed_text'][1]

'subject natur ga nomin enron methanol nomin follow natur ga requir methanol plant august mmbtu per day egpfc nomin follow natur ga requir mtbe plant morgan point august mmbtu per day'

In [8]:
len(train['processed_text'])

4136

In [10]:
train.head()

,Message_body,Label,processed_text
0,"Subject: meter 986296\r\nscherlyn , per our co...",0,subject meter scherlyn per convers meter month...
1,Subject: natural gas nomination for 08 / 00\r\...,0,subject natur ga nomin enron methanol nomin fo...
2,"Subject: cleburne outage\r\ngentlemen ,\r\ni w...",0,subject cleburn outag gentlemen want clarifi l...
3,"Subject: daren ,\r\nthe firm trading waha book...",0,subject daren firm trade waha book set intern ...
4,"Subject: re : mon , 2 feb 2004 03 : 16 : 16 - ...",1,subject mon feb page load imag show view messa...


In [12]:
featuresets = [(dialogue_act_features(train['processed_text'][i]), train['Label'][i])
              for i in range(len(train['processed_text']))]

In [13]:
classifier = nltk.NaiveBayesClassifier.train(featuresets)

In [14]:
testset = [(dialogue_act_features(test['processed_text'][i]), test['Label'][i])
              for i in range(len(test['processed_text']))]

In [16]:
print(nltk.classify.accuracy(classifier, testset))

0.9352657004830918


In [23]:
testset[1][0]

{'contains(subject)': True,
 'contains(feb)': True,
 'contains(intercompani)': True,
 'contains(accrual)': True,
 'contains(varianc)': True,
 'contains(list)': True,
 'contains(sitara)': True,
 'contains(deal)': True,
 'contains(volum)': True,
 'contains(one)': True,
 'contains(side)': True,
 'contains(purchas)': True,
 'contains(sale)': True,
 'contains(get)': True,
 'contains(accru)': True,
 'contains(goal)': True,
 'contains(bring)': True,
 'contains(everyon)': True,
 'contains(attent)': True,
 'contains(second)': True,
 'contains(see)': True,
 'contains(solut)': True,
 'contains(take)': True,
 'contains(simplist)': True,
 'contains(approach)': True,
 'contains(sinc)': True,
 'contains(nomin)': True,
 'contains(match)': True,
 'contains(month)': True,
 'contains(end)': True,
 'contains(wrong)': True,
 'contains(assumpt)': True,
 'contains(system)': True,
 'contains(limit)': True,
 'contains(free)': True,
 'contains(suggest)': True,
 'contains(idea)': True,
 'contains(resolut)': True

In [24]:
prediction = classifier.classify(testset[1][0])

In [28]:
predictions = [classifier.classify(testset[i][0]) for i in range(len(testset))]

In [29]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test['Label'], predictions, labels=None, sample_weight=None, normalize=None)

array([[672,  60],
       [  7, 296]], dtype=int64)

In [31]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(test['Label'], predictions,
                                average = 'binary')


(0.8314606741573034, 0.976897689768977, 0.898330804248862, None)